In [15]:
import numpy as np
import matplotlib as mpl

In [16]:
# Question 1
#: the diameter of the wire (x0), 
#the mean of the diameter of coil(x1)
#and the number of active coils (x2)

In [17]:
arrX = np.array([1, 2, 3])

In [18]:
#implementation of f(x)

In [31]:
def f(x, fCount=[0]):
    print(x[0] ** 2)*x[1]*(2 + x[2]))
    fCount[0]+=1
    return (fCount[0])

In [32]:
print(f(arrX))

AttributeError: 'numpy.int32' object has no attribute 'toString'

In [ ]:
#implementation of g1(x)

In [25]:
def g1(x):
    return (1 - (((x[1]**3)*x[2])/71785*(x[0]**4)))

In [26]:
print (g1(arrX))

0.9996656683151076


In [ ]:
#implementation of g2(x)

In [ ]:
def g2(x):
    return (())